In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn import linear_model, tree, ensemble

## Carregar Conjunto de Dados

In [2]:
dados_treino = pd.read_csv('train.csv')

# Remover linhas com valores nulos na variável alvo
dados_treino.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = dados_treino.SalePrice  # Variável alvo
dados_treino.drop(['SalePrice'], axis=1, inplace=True)  # Remover variável alvo dos dados de treino

# Remover colunas com valores nulos
dados_treino.drop(['LotFrontage', 'GarageYrBlt', 'MasVnrArea'], axis=1, inplace=True)

# Selecionar apenas colunas numéricas
colunas_numericas = [coluna for coluna in dados_treino.columns if dados_treino[coluna].dtype in ['int64', 'float64']]
X = dados_treino[colunas_numericas].copy()

print(f"Formato dos dados de entrada: {X.shape} e formato da variável alvo: {y.shape}")

Formato dos dados de entrada: (1460, 34) e formato da variável alvo: (1460,)


## Análise dos Dados
O conjunto de dados final contém 34 características e 1460 exemplos de treino. Nosso objetivo é prever o preço de venda das casas com base nos dados disponíveis.

## Avaliação do Modelo Usando KFold
Vamos usar a função **cross_val_score()** para avaliar o modelo por validação cruzada. Utilizaremos três modelos diferentes para análise. A média dos resultados será utilizada como desempenho final do modelo, baseado no erro quadrático médio (RMSE).

In [3]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
contador = 1
for indice_treino, indice_teste in kf.split(X, y):
    print(f'Fold:{contador}, Conjunto de treino: {len(indice_treino)}, Conjunto de teste:{len(indice_teste)}')
    contador += 1

Fold:1, Conjunto de treino: 1168, Conjunto de teste:292
Fold:2, Conjunto de treino: 1168, Conjunto de teste:292
Fold:3, Conjunto de treino: 1168, Conjunto de teste:292
Fold:4, Conjunto de treino: 1168, Conjunto de teste:292
Fold:5, Conjunto de treino: 1168, Conjunto de teste:292


In [4]:
def calcular_rmse(pontuacao):
    rmse = np.sqrt(-pontuacao)
    print(f'RMSE = {rmse:.2f}')

### Usando Regressão Linear

In [5]:
pontuacoes = cross_val_score(linear_model.LinearRegression(), X, y, cv=kf, scoring="neg_mean_squared_error")
print(f'Pontuações para cada fold: {pontuacoes}')
calcular_rmse(pontuacoes.mean())

Pontuações para cada fold: [-1.39334669e+09 -1.32533433e+09 -3.39493937e+09 -9.31045536e+08
 -7.16620849e+08]
RMSE = 39398.70


### Usando Regressor de Árvore de Decisão

In [6]:
pontuacoes = cross_val_score(tree.DecisionTreeRegressor(random_state=42), X, y, cv=kf, scoring="neg_mean_squared_error")
print(f'Pontuações para cada fold: {pontuacoes}')
calcular_rmse(pontuacoes.mean())

Pontuações para cada fold: [-2.28396934e+09 -1.70193863e+09 -2.50505513e+09 -1.48547479e+09
 -1.66691378e+09]
RMSE = 43916.63


### Usando Regressor de Floresta Aleatória

In [7]:
pontuacoes = cross_val_score(ensemble.RandomForestRegressor(random_state=42), X, y, cv=kf, scoring="neg_mean_squared_error")
print(f'Pontuações para cada fold: {pontuacoes}')
calcular_rmse(pontuacoes.mean())

Pontuações para cada fold: [-8.58316418e+08 -6.13821216e+08 -2.06121160e+09 -7.97273029e+08
 -5.68429309e+08]
RMSE = 31301.92
